In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset, Dataset
from trl import GRPOConfig, GRPOTrainer
from reward_model import FormatReward, AnswerReward

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 4096 # Can increase for longer reasoning traces
lora_rank = 64

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.75, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 04-07 20:52:15 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.14: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.676 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-7b-instruct-unsloth-bnb-4bit with actual GPU utilization = 72.1%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.68 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 10.94 GB. Also swap space = 5 GB.
INFO 04-07 20:52:24 config.py:549] This model supports multiple tasks: {'generate', 'embed', 'reward',

[W407 20:52:31.595028747 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 04-07 20:52:32 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 04-07 20:52:35 model_runner.py:1115] Loading model weights took 6.7252 GB
INFO 04-07 20:52:35 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-07 20:52:38 worker.py:267] Memory profiling takes 2.24 seconds
INFO 04-07 20:52:38 worker.py:267] the current vLLM instance can use total_gpu_memory (23.68GiB) x gpu_memory_utilization (0.72) = 17.07GiB
INFO 04-07 20:52:38 worker.py:267] model weights take 6.73GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.25GiB; the rest of the memory reserved for KV Cache is 9.05GiB.
INFO 04-07 20:52:38 executor_base.py:111] # cuda blocks: 10588, # CPU blocks: 5851
INFO 04-07 20:52:38 executor_base.py:116] Maximum concurrency for 4096 tokens per request: 41.36x
INFO 04-07 20:52:40 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error o

Capturing CUDA graph shapes: 100%|██████████| 31/31 [00:15<00:00,  1.96it/s]

INFO 04-07 20:52:56 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.67 GiB
INFO 04-07 20:52:56 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 20.35 seconds



Unsloth 2025.3.14 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
SYSTEM_PROMPT = """
Your task is to solve the following Sudoku puzzle.
Empty cells are represented by blank spaces. Fill in all missing numbers such that each row, column, and 3×3 subgrid contains the digits 1 through 9 exactly once.
Put the fully solved Sudoku grid in the same format as the input, including dividers in answer section.
Please respond in the following format:
<reasoning>
...
</reasoning>

<answer>
...
</answer>
"""

def get_sudoku_probelms() -> Dataset:
    data = load_dataset("csv", data_files="problems/train.csv")["train"] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x["pretty_puzzle"]}
        ]
    }) # type: ignore
    return data

dataset = get_sudoku_probelms()

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [5]:
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 3e-5,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 1000,
    max_completion_length = 1000,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [6]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        FormatReward.tag_reward_func,
        FormatReward.tag_order_reward_func,
        AnswerReward.answer_format_reward_func,
        AnswerReward.correct_answer_reward_func,
        AnswerReward.clue_preservation_reward_func,
        AnswerReward.rule_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 161,480,704/7,000,000,000 (2.31% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / tag_reward_func,rewards / tag_order_reward_func,rewards / answer_format_reward_func,rewards / correct_answer_reward_func,rewards / clue_preservation_reward_func,rewards / rule_reward_func
1,0.000000,1.091435,1.138640,841.000000,0.000000,0.406250,0.250000,0.435185,0.000000,0.000000,0.000000
2,0.000000,1.901200,1.279021,821.000000,0.000000,0.718750,0.500000,0.538931,0.000000,0.106481,0.037037
3,0.000000,1.609778,1.404555,865.875000,0.000433,0.437500,0.250000,0.618266,0.000000,0.192901,0.111111
4,0.000000,1.118722,1.190576,939.250000,0.000490,0.343750,0.125000,0.540404,0.000000,0.081790,0.027778
5,0.000000,1.333930,1.695771,954.000000,0.000347,0.281250,0.250000,0.537247,0.000000,0.200617,0.064815
6,0.000000,1.606482,1.198932,908.750000,0.000503,0.562500,0.500000,0.543981,0.000000,0.000000,0.000000
7,0.000000,1.881383,1.867186,870.250000,0.000646,0.500000,0.375000,0.583544,0.000000,0.283951,0.138889
8,0.000000,1.647166,1.599199,905.750000,0.000898,0.500000,0.250000,0.617845,0.000000,0.182099,0.097222
9,0.000100,3.708859,1.311353,682.125000,0.003745,0.906250,0.875000,0.797980,0.000000,0.750000,0.379630
10,0.000100,1.700793,1.312584,842.875000,0.001707,0.656250,0.375000,0.541456,0.000000,0.086420,0.041667


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=50, training_loss=0.0006948188267504562, metrics={'train_runtime': 2522.9299, 'train_samples_per_second': 0.159, 'train_steps_per_second': 0.02, 'total_flos': 0.0, 'train_loss': 0.0006948188267504562})

In [7]:
model.save_lora("grpo_saved_lora")

In [8]:
content = dataset[2]["pretty_puzzle"]

text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : content},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

print(output)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.22s/it, est. speed input: 51.00 toks/s, output: 59.82 toks/s]

<reasoning>
1. Start by filling in the first row. Since 2 and 5 are already in the first row, the missing number is 3.
2. Continue this process for the rest of the grid, filling in numbers based on the given numbers and the rules of Sudoku.
3. Use the process of elimination and checking the possibilities for each cell to solve the puzzle.
</reasoning>

<answer>
  2 5 3 | 7 4 9 | 1 6 8  
  7 4 8 | 5 6 3 | 9 8 2  
  3 1 6 | 8 1 2 | 7 5 4  
- - - + - - - + - - -  
  1 9 7 | 8 5 4 | 6 3 2  
  5 8 2 | 4 3 6 | 3 1 8  
  6 4 3 | 2 7 1 | 5 9 7  
- - - + - - - + - - -  
  9 8 4 | 1 7 5 | 3 2 6  
  6 2 5 | 3 8 7 | 4 4 9  
  1 3 9 | 2 6 8 | 7 1 5  
</answer>


In [9]:
print(content)

  2 5 |   4   | 1    
  7 4 | 5 6 3 | 9 8 2
3   6 | 8 1 2 |   5 4
- - - + - - - + - - -
      |   8   | 6    
  5   | 4   6 | 2 1 8
6 4   | 2   1 | 5   7
- - - + - - - + - - -
9 8   |       |   2  
  6 2 | 3     |   9  
  1 3 |   2 9 | 8 7  
